In [1]:
import baostock as bs
import pandas as pd
import time
import sqlite3

In [3]:
indices={
    'sh.000001':'上证指数',
    'sz.399001':'深证成指',
    'sz.399106':'深证综指',
    'sh.000300':'沪深300',
    'sz.399006':'创业板',
    'sh.000016':'上证50'
}
# indices.keys()
# indices.values()

In [41]:
def select(sql):
    if str(sql).lower().find('select') < 0:
        raise ValueError('Select Statement only!!! For args sql = ', sql)

    # 获取数据库连接，数据库文件不存在时，会自动创建数据库文件
    conn = sqlite3.connect('stock.db')
    # 获取游标
    cursor = conn.cursor()
    cursor.execute(sql)
    res = cursor.fetchall()
    cursor.close()
    conn.close()
    return res

In [5]:
def executeUpdate(sql):
    # 获取数据库连接，数据库文件不存在时，会自动创建数据库文件
    conn = sqlite3.connect('stock.db')
    # 获取游标
    cursor = conn.cursor()
    cursor.execute(sql)
    cursor.close()
    conn.commit()
    conn.close()

In [56]:
# 获取指定日期范围内的指数记录，并插入数据库
# code 指数代码
# start_date 开始日期
# end_date 结束日期
def dump_zs001(code, start_date, end_date):
    # 指数数据list
    zs_data=[]
    res = bs.query_history_k_data_plus(code, "date,code,open,high,low,close,preclose,volume,amount,pctChg",
        start_date, end_date, frequency="d")
    # 提取数据到data_list
    if res.error_code == '0':
        while res.next():
            zs_data.append(res.get_row_data())

    print(start_date, code, '获取记录数：',len(zs_data))
    # print(zs_data)
    # 创建指数记录表
    '''cursor.execute(
        "create table market_index(
            date comment '日期',
            code comment '指数代码',
            open comment '开盘',
            high comment '最高',
            low comment '最低',
            close comment '收盘',
            preclose comment '昨收',
            volume comment '成交数量',
            amount comment '成交金额',
            pctChg comment '涨跌幅',
            turn comment '换手率')")
    '''

    if len(zs_data)>0:

        # 获取数据库连接，数据库文件不存在时，会自动创建数据库文件
        conn = sqlite3.connect('stock.db')

        # 获取游标
        cursor = conn.cursor()
        # 插入数据库
        cres=cursor.executemany('insert into market_index(date,code,open,high,low,close,preclose,volume,amount,pctChg,turn) values(?,?,?,?,?,?,?,?,?,?,?)',
             zs_data)
        # print('cres', cres)
        # 输出提交记录数，并提交
        print(start_date, code, '插入记录数：', cursor.rowcount)
        # 先关闭游标
        # cursor.colse() # 为什么报错：cursor对象没有close()属性？？？
        # 再提交
        conn.commit()
        conn.close()

    print("记录dump完成")

In [72]:
# 市场指数表，增加“换手率”字段
# executeUpdate("ALTER TABLE market_index ADD turn COMMENT '换手率'")

In [78]:
# 补充“换手率”字段
# code 指数代码
# start_date 开始日期
# end_date 结束日期
def dump_zsturn001(code, start_date, end_date):
    # 指数数据list
    zs_data=[]
    res = bs.query_history_k_data_plus(code, "turn,date,code", start_date, end_date, frequency="d")
    # 提取数据到data_list
    if res.error_code == '0':
        while res.next():
            zs_data.append(res.get_row_data())

    print(start_date, code, '获取记录数：',len(zs_data))

    if len(zs_data)>0:

        # 获取数据库连接，数据库文件不存在时，会自动创建数据库文件
        conn = sqlite3.connect('stock.db')

        # 获取游标
        cursor = conn.cursor()
        # 插入数据库
        cres=cursor.executemany('update market_index set turn=? where date=? and code=?', zs_data)
        # 输出提交记录数，并提交
        print(start_date, code, '插入记录数：', cursor.rowcount)
        # 先关闭游标
        # cursor.colse() # 为什么报错：cursor对象没有close()属性？？？
        # 再提交
        conn.commit()
        conn.close()

    print("补充‘换手率’完成")

In [54]:
# executeUpdate('DELETE FROM market_index')

In [83]:
select("select * from market_index where date='2021-01-27'")

[('2021-01-27',
  'sh.000001',
  3567.5485,
  3578.7968,
  3546.492,
  3573.3412,
  3569.429,
  26410716100,
  396270389618.8,
  0.109603,
  0.721165),
 ('2021-01-27',
  'sz.399001',
  15328.8694,
  15446.8202,
  15107.0425,
  15413.8432,
  15352.4196,
  33837216736,
  526877852706.76,
  0.400091,
  1.688154),
 ('2021-01-27',
  'sz.399106',
  2409.0745,
  2424.8561,
  2379.208,
  2420.9157,
  2414.1605,
  33837216736,
  526877852706.76,
  0.279816,
  1.805976),
 ('2021-01-27',
  'sh.000300',
  5505.7708,
  5534.9928,
  5449.6385,
  5528.0034,
  5512.9678,
  16019084100,
  376892605839.1,
  0.272732,
  0.580319),
 ('2021-01-27',
  'sz.399006',
  3269.773,
  3296.163,
  3196.0881,
  3281.0305,
  3258.3553,
  9148409385,
  182490438944.83,
  0.695909,
  2.349896),
 ('2021-01-27',
  'sh.000016',
  3818.4218,
  3831.6177,
  3790.8094,
  3818.2682,
  3813.2902,
  2870605400,
  92127165903,
  0.130543,
  0.227199)]

In [62]:
select("SELECT count(*) FROM market_index where date like '2021%' and code='sh.000001'")

[(18,)]

In [63]:
select("SELECT date FROM market_index where date like '2021%' and code='sh.000001'")

[('2021-01-04',),
 ('2021-01-05',),
 ('2021-01-06',),
 ('2021-01-07',),
 ('2021-01-08',),
 ('2021-01-11',),
 ('2021-01-12',),
 ('2021-01-13',),
 ('2021-01-14',),
 ('2021-01-15',),
 ('2021-01-18',),
 ('2021-01-19',),
 ('2021-01-20',),
 ('2021-01-21',),
 ('2021-01-22',),
 ('2021-01-25',),
 ('2021-01-26',),
 ('2021-01-27',)]

In [79]:
# 测试--dump市场指数数据
# dump_zs001('sh.000001', '2000-01-01', '2000-12-31')
# 测试--补充换手率
# dump_zsturn001('sh.000001', '2021-01-01', '2021-12-31')

2021-01-01 sh.000001 获取记录数： 18
2021-01-01 sh.000001 插入记录数： 18


In [82]:
lg = bs.login()
print(lg.error_code)
print(lg.error_msg)

# 逐个指数获取市场指数记录，并插入数据库
# 2000-01-01~2021-01-27
#for idx in indices.keys():
#    for yr in range(0, 22):
#        dump_zs001(idx, '20%02d-01-01' % yr, '20%02d-12-31' % yr)

# 逐个指数获取市场指数记录，补充“换手率”
# 2000-01-01~2021-01-27
for idx in indices.keys():
    for yr in range(0, 22):
        dump_zsturn001(idx, '20%02d-01-01' % yr, '20%02d-12-31' % yr)

2000-01-01 sh.000001 获取记录数： 239
2000-01-01 sh.000001 插入记录数： 239
2001-01-01 sh.000001 获取记录数： 240
2001-01-01 sh.000001 插入记录数： 240
2002-01-01 sh.000001 获取记录数： 237
2002-01-01 sh.000001 插入记录数： 237
2003-01-01 sh.000001 获取记录数： 241
2003-01-01 sh.000001 插入记录数： 241
2004-01-01 sh.000001 获取记录数： 243
2004-01-01 sh.000001 插入记录数： 243
2005-01-01 sh.000001 获取记录数： 242
2005-01-01 sh.000001 插入记录数： 242
2006-01-01 sh.000001 获取记录数： 241
2006-01-01 sh.000001 插入记录数： 241
2007-01-01 sh.000001 获取记录数： 242
2007-01-01 sh.000001 插入记录数： 242
2008-01-01 sh.000001 获取记录数： 246
2008-01-01 sh.000001 插入记录数： 246
2009-01-01 sh.000001 获取记录数： 244
2009-01-01 sh.000001 插入记录数： 244
2010-01-01 sh.000001 获取记录数： 242
2010-01-01 sh.000001 插入记录数： 242
2011-01-01 sh.000001 获取记录数： 244
2011-01-01 sh.000001 插入记录数： 244
2012-01-01 sh.000001 获取记录数： 243
2012-01-01 sh.000001 插入记录数： 243
2013-01-01 sh.000001 获取记录数： 238
2013-01-01 sh.000001 插入记录数： 238
2014-01-01 sh.000001 获取记录数： 245
2014-01-01 sh.000001 插入记录数： 245
2015-01-01 sh.000001 获取记录数： 244
2015-01-

In [ ]:
# 当前日期yyyy-MM-dd
# time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) 
sdate = time.strftime("%Y-%m-%d", time.localtime())
lg = bs.login()
print(lg.error_code)
print(lg.error_msg)


In [24]:


# 读取指数详情
for idx in indices.keys():
    dump_zs001(idx, sdate, sdate)

# 上证指数(0000001)
get_zs('sh.000001')

# 深证成指(399001)
get_zs('sz.399001')

# 深证综指(399106)
get_zs('sz.399106')

# 沪深300
get_zs('sh.000300')

# 创业板
get_zs('sz.399006')

# 上证50
get_zs('sh.000016')

# 科创50
#get_zs('sh.000688')

zsdf = pd.DataFrame(data=data_list, columns=[
     'date',# 日期
     'code',# 代码
     'open',# 开盘
     'high',# 最高
     'low',# 最低
     'close',# 收
     'preclose',# 昨收
     'volume',# 成交数量（股）
     'amount',# 成交金额（元）
     'pctChg',# 涨跌幅
     'turn',# 换手率
    ])

if zsdf.size > 0:
    sh_amount=float(zsdf['amount'][0])
    sz_amount=float(zsdf['amount'][1])
    sz_t_amount=float(zsdf['amount'][2])
    z300_amount=float(zsdf['amount'][3])
    c_amount=float(zsdf['amount'][4])
    z50_amount=float(zsdf['amount'][5])
#    kc_amount=float(zsdf['amount'][6])

    change_percent = zsdf['pctChg']

    total_amount = (sh_amount + sz_amount) / 1e8
    print('两市共成交：%.2f 亿元' % total_amount)
    print('上证指数涨幅：%.2f %%,成交量：%.2f 亿元' % (float(change_percent[0]), sh_amount / 1e8))
    print('深证综指涨幅：%.2f %%,成交量：%.2f 亿元' % (float(change_percent[2]), sz_t_amount / 1e8))
    print('深证成指涨幅：%.2f %%,成交量：%.2f 亿元' % (float(change_percent[1]), sz_amount / 1e8))
    print('沪深300涨幅：%.2f %%,成交量：%.2f 亿元' % (float(change_percent[3]), z300_amount / 1e8))
    print('创业板涨幅：%.2f %%,成交量：%.2f 亿元' % (float(change_percent[4]), c_amount / 1e8))
    print('上证50涨幅：%.2f %%,成交量：%.2f 亿元' % (float(change_percent[5]), z50_amount / 1e8))
#    print('科创50涨幅：%.2f %%,成交量：%.2f 亿元' % (float(change_percent[6]), kc_amount / 1e8))
else:
    print('get Data failed')

success
success
success
success
success
success
两市共成交：12083.02 亿元
上证指数涨幅：0.48 %,成交量：5272.57 亿元
深证综指涨幅：0.27 %,成交量：6810.45 亿元
深证成指涨幅：0.52 %,成交量：6810.45 亿元
沪深300涨幅：1.01 %,成交量：5081.67 亿元
创业板涨幅：-0.09 %,成交量：2372.84 亿元
上证50涨幅：1.35 %,成交量：1363.41 亿元


### 关于BaoStock
1、BaoStock的登录接口调用似乎限制了接口获取数据的最后日期，只能获取登录日期之前的数据。juypter notebook使用昨日的登录结果，调用接口时得不到今天的数据。

### 2021-01-28 
- 2000-01-01至2021-01-27市场指数落库`market_index`
- 补充换手率